In [1]:
# Оценка
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

# Пайплайн
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

# Данные
import os
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder
from sklearn.preprocessing import RobustScaler

In [2]:
# Препроцессоры
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """ Предобработчик данных """
    def __init__(self, cat_features, transform_train=True):
        self.transform_train = transform_train
        self.cat_features = cat_features

        self.bin_encoder = BinaryEncoder(cols=cat_features)
        self.robust_scaler = RobustScaler()

    def fit(self, X, y=None):
        # Создаём копию датасета
        X_ = X.copy()

        # Временные фичи
        # X_['day_sin'] = np.sin(np.pi * 2 * X_['День недели 2'] / 6)
        # X_['day_cos'] = np.cos(np.pi * 2 * X_['День недели 2'] / 6)

        # Категориальные фичи
        X_ = self.bin_encoder.fit_transform(X_)

        # Нумерация фич для LGBM (не принимает JSON символы)
        X_.columns = [num for num in range(0, len(X_.columns))]

        # Масштабирование
        self.robust_scaler.fit(X_)

        return self
    
    def transform(self, X):
        # Создаём копию датасета
        X_ = X.copy()

        # Временные фичи
        # X_['day_sin'] = np.sin(np.pi * 2 * X_['День недели 2'] / 6)
        # X_['day_cos'] = np.cos(np.pi * 2 * X_['День недели 2'] / 6)

        # Категориальные фичи
        X_ = self.bin_encoder.transform(X_)

        # Нумерация фич для LGBM (не принимает JSON символы)
        X_.columns = [num for num in range(0, len(X_.columns))]

        # Масштабирование
        X_ = self.robust_scaler.transform(X_)

        return X_


In [3]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')

# Загрузка
train_df = pd.read_csv(TRAIN_DATASET)
balanced_df = pd.read_csv(BALANCED_DATASET, index_col=0)
test_df = pd.read_csv(TEST_DATASET)

# Обрезка
# first_negatives = train_df[train_df['y'] == 0][:train_df[train_df['y'] == 1]['y'].count()]
# train_df = pd.concat([train_df[train_df['y'] == 1], first_negatives])

X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Список фич на дроп
features_to_drop = [
    'Изменение позиции заказа на закупку: изменение даты поставки на бумаге',
    'Количество', 'Категорийный менеджер', 'Завод', 'Материал',
    'Отмена полного деблокирования заказа на закупку'
    ]

# Список фич на удержание
# features_to_keep = ['Поставщик', 'Длительность', 'Закупочная организация', 'Балансовая единица',
#                     'ЕИ', 'Сумма', 'Вариант поставки', 'Количество обработчиков 7', 
#                     'Количество обработчиков 15', 'Количество обработчиков 30']
# features_to_drop = X.columns[~X.columns.isin(features_to_keep)]

# Категориальные (пропускаются через BinaryEncoder)
cat_features = [
    'Поставщик', 'Закупочная организация', 
    'Балансовая единица', 'ЕИ', 'Вариант поставки'
    ]

# Выкидываем фичи
X_train = X_train.drop(features_to_drop, axis=1)
X_test = X_test.drop(features_to_drop, axis=1)
test_df = test_df.drop(features_to_drop, axis=1)

In [8]:
# Параметры классификатора
best_params = {
    'learning_rate': 0.3, 
    'n_estimators': 1000,
    'max_depth': 8,
    'max_bin': 128,
    'num_leaves': 128,
    'reg_lambda': 0.3
    }

# Модель
data_preprocessor = DataPreprocessor(cat_features)
model = LGBMClassifier(
    **best_params,
    n_jobs=-1,
    force_col_wise=True,
    verbose=-1
)

pipeline = Pipeline([
    ('data_preproc', data_preprocessor),
    ('model', model)
])

pipeline.fit(X_train, y_train)
print(f"CV Score: {cross_val_score(estimator=pipeline, X=X_train, y=y_train, cv=StratifiedKFold(), n_jobs=-1).mean()}")
print(f"TEST Score: {f1_score(y_test, pipeline.predict(X_test), average='macro')}")

CV Score: 0.947116049382716
TEST Score: 0.8938672029046602


In [9]:
# Сабмит
preds = pipeline.predict(test_df)
submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})
submit_df.to_csv('submission.csv', index=False)